In [1]:
import subprocess 
import json
import os
from dotenv import load_dotenv
from Constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account  

In [2]:
#Loading the env file
load_dotenv()
mnemonic = os.getenv('mnemonic')


In [3]:
# Nodes runing with POW
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1.8545"))


In [4]:
def derive_wallets(mnemonic, coin, numderive):
    # Use the subprocess library to call the php file script from Python
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    keys =json.loads(output)
    return keys

In [5]:
#Setting dictionary of coins to be used in the wallet
coins = {"eth", "btc-test"}
numderive = 3

In [6]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [7]:
eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']


print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))

"0x234b4a92062826335b6a81c54d0826f82d0d36db009936949a9d3923072510d1"
"cNGou8YCa1Zdio231e1z1JwGHMyfGgfUEDaerYT8SmmPL2zXixtG"


In [8]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc-test": [
        {
            "address": "n36oMUSWYJeX9RdmQcd8PtuzV3qCs8Bmhn",
            "index": 0,
            "path": "m/44'/1'/0'/0/0",
            "privkey": "cNGou8YCa1Zdio231e1z1JwGHMyfGgfUEDaerYT8SmmPL2zXixtG",
            "pubkey": "027d34df57f7e0659fe779fa3d55ae56de3afd3fe88e129efe59a5221666b1a14f",
            "pubkeyhash": "ecc1521026a9045ea79ba6aa380f07ab316f898c",
            "xprv": "tprv8kswE75Rg1Gq1DUXbuEy5JRFV69uURiCXautCtGKFDHjRe4uzNLx8wioA4kZCYMJwxzGyCPfLEP3XXL6k9gtYigoqn5q37mvUKhShU9cFsb",
            "xpub": "tpubDHZyNX7fpNxVtgWKVYuZUi5N47fqdku76tWfVQJcfV68G8KgcmAYKSLfLCVWMRV6Qs3cc2crP8eS5dHG4tvX8NFsXSrSW7gzmxGZemc6tPr"
        },
        {
            "address": "msvie32QhBQKeywrAsVB53gwffUfAbRdk7",
            "index": 1,
            "path": "m/44'/1'/0'/0/1",
            "privkey": "cSAzrdm47YrAUfpM1n4bGut1zJoCoqqLpytPm6xysAhbPZb4fazT",
            "pubkey": "03ece1f5f196db3dc7407ecaca63904d43513a62b7d8240baf85546039136b0588",
            "pubkeyh

In [9]:
def priv_key_to_account(coin,priv_key):
    print(coin)
    print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [10]:
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)

btc-test
cNGou8YCa1Zdio231e1z1JwGHMyfGgfUEDaerYT8SmmPL2zXixtG
eth
0x234b4a92062826335b6a81c54d0826f82d0d36db009936949a9d3923072510d1


In [11]:
print(eth_acc)
print(btc_acc)

<PrivateKeyTestnet: n36oMUSWYJeX9RdmQcd8PtuzV3qCs8Bmhn>


In [12]:
def create_tx(coin, account, recipient, amount):
    # create the raw, unsigned transaction that contains all metadata needed to transact 
    global trx_data
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        trx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)]) 

In [13]:
def send_tx(coin, account, recipient, amount):
    # call create_trx, sign the transaction, then send it to the designated network
    if coin == "eth": 
        trx_eth = create_tx(coin,account, recipient, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trx_btctest= create_tx(coin,account,recipient,amount)
        sign_tx_btctest = account.sign_transaction(trx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_tx_btctest)       
        return sign_tx_btctest

In [14]:
create_tx(BTCTEST,btc_acc,'n36oMUSWYJeX9RdmQcd8PtuzV3qCs8Bmhn', 0.00001)

'{"unspents":[{"amount":100000,"confirmations":0,"script":"76a914ecc1521026a9045ea79ba6aa380f07ab316f898c88ac","txid":"e44315ce88108798ae0a9e3c9f6913349817afe53e5f901db553d9e86ef1f22d","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["n36oMUSWYJeX9RdmQcd8PtuzV3qCs8Bmhn",1000],["n36oMUSWYJeX9RdmQcd8PtuzV3qCs8Bmhn",75948]]}'

In [20]:
send_tx(BTCTEST,btc_acc,'n36oMUSWYJeX9RdmQcd8PtuzV3qCs8Bmhn',0.00001)

'0100000001b5cbccbe22b4d3d2256c3697aeffaebab6db185fca8c61b808bf37cd5e56c63b010000006a473044022062a572530b42529f66d2c558580fa77cdd600dacc8083af909536eb5f3790529022079e40ae7a70ee661a7267333474b1dab238b724ec31f738d1fa6b2ef046963300121027d34df57f7e0659fe779fa3d55ae56de3afd3fe88e129efe59a5221666b1a14fffffffff02e8030000000000001976a914ecc1521026a9045ea79ba6aa380f07ab316f898c88acb8ca0000000000001976a914ecc1521026a9045ea79ba6aa380f07ab316f898c88ac00000000'

In [16]:
from web3.middleware import geth_poa_middleware

w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [17]:
from web3 import Web3, HTTPProvider

In [18]:
#connecting to HTTP with address pk
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0x5619d0cce54919511eb9f3b678de07bd346044acee0bdd6cdf4497ba45e03f81"))


In [21]:
w3.isConnected()

False

In [19]:
#checking the Block Number
w3.eth.blockNumber

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8545): Max retries exceeded with url: /0x5619d0cce54919511eb9f3b678de07bd346044acee0bdd6cdf4497ba45e03f81 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002324F9E91C0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
# double check if  I am connected to blockchain. 
w3.isConnected()

In [ ]:
w3.eth.getBalance("0x2Ad0FD35E0Df759985828a0E9597b98eb7caAE5e")

In [ ]:
create_tx(ETH,eth_acc,"0x2Ad0FD35E0Df759985828a0E9597b98eb7caAE5e", 1000)

In [ ]:
send_tx(ETH, eth_acc,"0x2Ad0FD35E0Df759985828a0E9597b98eb7caAE5e", 1000)

In [ ]:
w3.eth.getBalance("0x2Ad0FD35E0Df759985828a0E9597b98eb7caAE5e")